In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from enum import Enum
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from itertools import product
import sys
from pathlib import Path

In [2]:
import os, sys
sys.path.append(os.path.abspath("../../etc/"))
import config

In [3]:
df_raw = pd.read_csv('./data/raw_transformation_01.csv', index_col=0)
df_processed = pd.read_csv('./data/01_binding_labels.csv', index_col=0)

In [4]:
biological_sequence_list = config.EXTRACTABLE_BIOSEQUENCE_FEATURES

In [5]:
get_length = lambda col: df_raw[col].str.len().fillna(0).astype(int)
lowercase_col_names = lambda col: str.lower(col).replace(' ', '_')

AMINO_ACIDS = config.AMINO_ACID_ALPHABETS

CONJOINT_TRIADS = {
    'A': '1', 'G': '1', 'V': '1',             # Aliphatic
    'I': '2', 'L': '2', 'F': '2', 'P': '2',   # Large Aliphatic
    'Y': '3', 'M': '3', 'T': '3', 'S': '3',   # Polar
    'H': '4', 'N': '4', 'Q': '4', 'W': '4',   # Aromatic/Neutral
    'R': '5', 'K': '5',                       # Positive Charge
    'D': '6', 'E': '6',                       # Negative Charge
    'C': '7'                                  # Cysteine (Special)
}

#Pre-generate the list of all 343 possible triads
TRIAD_NAMES = ["".join(t) for t in product("1234567", repeat=3)]
KMER3_NAMES = ["".join(t) for t in product(config.AMINO_ACID_ALPHABETS, repeat=3)]

def get_conjoint_triads_fast(df, col):
    # Vectorized translation of sequence to groups
    translation_table = str.maketrans(CONJOINT_TRIADS)
    # We remove non-standard AAs by ignoring anything not in our 7 groups
    encoded_series = df[col].fillna("").str.translate(translation_table)
    
    # Prepare the CountVectorizer for 3-character "words"
    # analyzer='char' and ngram_range=(3,3) mimics the triad sliding window
    
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3), vocabulary=TRIAD_NAMES)
    triad_matrix = vectorizer.transform(encoded_series)
    lengths = encoded_series.str.len() - 2
    lengths = lengths.clip(lower=1) # Prevent div by zero
    triad_df = pd.DataFrame(
        triad_matrix.toarray() / lengths.values[:, None], # type: ignore
        columns=[f"{col}_CTD_{t}" for t in TRIAD_NAMES],
        index=df.index
    )
    
    return triad_df



def get_3kmer_motif_triads_fast(df: pd.DataFrame, col: str):
    # We remove non-standard AAs by ignoring anything not in our 7 groups
    encoded_series = df[col].fillna("")
    
    # Prepare the CountVectorizer for 3-character "words"
    # analyzer='char' and ngram_range=(3,3) mimics the triad sliding window
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3), vocabulary=KMER3_NAMES, lowercase=False)
    triad_matrix = vectorizer.transform(encoded_series)
    lengths = encoded_series.str.len() - 2
    lengths = lengths.clip(lower=1) # Prevent div by zero
    
    triad_df = pd.DataFrame(
        triad_matrix.toarray() / lengths.values[:, None], # type: ignore
        columns=[f"{col}_3kmer_{t}" for t in KMER3_NAMES],
        index=df.index,
        
    )
    
    return triad_df

feature_conjoint_triads_dfs = []
feature_kmer_3_dfs = []


for seq in biological_sequence_list:
    # Get CTD info as its own DF
    ctd_df = get_conjoint_triads_fast(df_raw, seq)
    feature_conjoint_triads_dfs.append(ctd_df)
    
    # kmer3_df = get_3kmer_motif_triads_fast(df_raw, seq)
    # feature_kmer_3_dfs.append(kmer3_df)
    
df_processed_conjoint = pd.concat([df_processed] + feature_conjoint_triads_dfs, axis=1)
# df_processed_motif = pd.concat([df_processed] + feature_kmer_3_dfs, axis=1)

In [6]:
df_processed_conjoint.describe()

,is_binding_SARS-CoV2_WT,is_neutral_SARS-CoV2_WT,is_nanobody,CDRH3_CTD_111,CDRH3_CTD_112,CDRH3_CTD_113,CDRH3_CTD_114,CDRH3_CTD_115,CDRH3_CTD_116,CDRH3_CTD_117,...,VHorVHH_CTD_765,VHorVHH_CTD_766,VHorVHH_CTD_767,VHorVHH_CTD_771,VHorVHH_CTD_772,VHorVHH_CTD_773,VHorVHH_CTD_774,VHorVHH_CTD_775,VHorVHH_CTD_776,VHorVHH_CTD_777
count,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,17918.000000,...,17918.000000,17918.000000,1.791800e+04,17918.000000,17918.000000,17918.000000,1.791800e+04,17918.000000,17918.0,17918.0
mean,1.021933,0.879842,0.067251,0.018201,0.012793,0.016926,0.005191,0.004098,0.007180,0.000492,...,0.000004,0.000005,5.694878e-07,0.000005,0.000001,0.000005,3.341904e-07,0.000001,0.0,0.0
std,0.302764,0.765030,0.250463,0.048646,0.030908,0.034747,0.020389,0.018082,0.023732,0.005668,...,0.000182,0.000204,7.623057e-05,0.000196,0.000108,0.000194,4.473411e-05,0.000087,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0
50%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0
75%,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0
max,2.000000,2.000000,1.000000,0.800000,0.400000,0.333333,0.333333,0.250000,0.250000,0.125000,...,0.008333,0.008475,1.020408e-02,0.008621,0.008333,0.008475,5.988024e-03,0.007874,0.0,0.0


In [7]:
# df_processed_motif.describe()

In [9]:
OUTFILE_PATH_CONJOINT = './data/03_conjoint_triads_processed_features.csv'
# OUTFILE_PATH_MOTIF = './data/03_kmer3_processed_features.csv'
df_processed_conjoint.drop(columns=[config.BINDING_TARGET,config.NEUTRAL_TARGET,config.IS_NANOBODY_COL]).to_csv(OUTFILE_PATH_CONJOINT)
# df_processed_motif.drop(columns=[config.BINDING_TARGET,config.NEUTRAL_TARGET,config.IS_NANOBODY_COL]).to_csv(OUTFILE_PATH_MOTIF)